In [1]:
import pandas as pd
import re

In [2]:
#importing the dataset into a DataFrame
df = pd.read_csv('mbti_1.csv')

In [3]:
#showing the first 5 rows of the dataset
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [4]:
#showing dataset infos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 135.7+ KB


In [5]:
#removing URLs and punctuation from dataset
for index in df.index.values:
    df.posts.iloc[index] = ' '.join(df.posts.iloc[index].split('|||'))
    df.posts.iloc[index] = re.sub(r"http\S+","",df.posts.iloc[index])
    df.posts.iloc[index] = re.sub(r"[-/@.?!_,:;()|0-9]","",df.posts.iloc[index])
    df.posts.iloc[index] = ' '.join(df.posts.iloc[index].split('  '))
"""
saving the clean dataset into a csv file
df.to_csv('mbti_clean_dataset.csv',index=False)
"""

"\nsaving the clean dataset into a csv file\ndf.to_csv('mbti_clean_dataset.csv',index=False)\n"

In [6]:
#identifying the different classes of users in the dataset
labels = df.type.unique()
labels

array(['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'],
      dtype=object)

In [7]:
#mapping personality types with their numberical representation
labels2 = []
label_rep = {}
for index,labels in enumerate(labels):
    label_rep[labels] = index
    labels2.append(labels)
labels2

['INFJ',
 'ENTP',
 'INTP',
 'INTJ',
 'ENTJ',
 'ENFJ',
 'INFP',
 'ENFP',
 'ISFP',
 'ISTP',
 'ISFJ',
 'ISTJ',
 'ESTP',
 'ESFP',
 'ESTJ',
 'ESFJ']

In [8]:
label_rep

{'INFJ': 0,
 'ENTP': 1,
 'INTP': 2,
 'INTJ': 3,
 'ENTJ': 4,
 'ENFJ': 5,
 'INFP': 6,
 'ENFP': 7,
 'ISFP': 8,
 'ISTP': 9,
 'ISFJ': 10,
 'ISTJ': 11,
 'ESTP': 12,
 'ESFP': 13,
 'ESTJ': 14,
 'ESFJ': 15}

In [9]:
#replacing each personality type with its numerical representation
df['label'] = df.type.replace(label_rep)
df.head(10)

,type,posts,label
0,INFJ,' enfp and intj moments sportscenter not top ...,0
1,ENTP,'I'm finding the lack of me in these posts ver...,1
2,INTP,'Good one Of course to which I say I know th...,2
3,INTJ,'Dear INTP I enjoyed our conversation the oth...,3
4,ENTJ,'You're fired That's another silly misconcepti...,4
5,INTJ,' Science is not perfect No scientist claims t...,3
6,INFJ,'No I can't draw on my own nails haha Those we...,0
7,INTJ,'I tend to build up a collection of things on ...,3
8,INFJ,I'm not sure that's a good question The distin...,0
9,INTP,' I'm in this position where I have to actuall...,2


In [10]:
# 저장하기
df.to_csv("mbti_cleaned_datasets.csv")

In [11]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import keras
from tqdm import tqdm
import pickle
from keras.models import Model
import keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import itertools
from keras.models import load_model
from sklearn.utils import shuffle
from transformers.models.bert.modeling_bert import BertModel,BertForMaskedLM
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig

In [12]:

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [13]:
data = pd.read_csv('mbti_cleaned_datasets.csv')

In [14]:
data.head()

,Unnamed: 0,type,posts,label
0,0,INFJ,' enfp and intj moments sportscenter not top ...,0
1,1,ENTP,'I'm finding the lack of me in these posts ver...,1
2,2,INTP,'Good one Of course to which I say I know th...,2
3,3,INTJ,'Dear INTP I enjoyed our conversation the oth...,3
4,4,ENTJ,'You're fired That's another silly misconcepti...,4


In [15]:
print('File has {} rows and {} columns'.format(data.shape[0],data.shape[1]))

File has 8675 rows and 4 columns


In [16]:
# Select required columns
data = data[['posts', 'label']]

In [17]:
data.head()

,posts,label
0,' enfp and intj moments sportscenter not top ...,0
1,'I'm finding the lack of me in these posts ver...,1
2,'Good one Of course to which I say I know th...,2
3,'Dear INTP I enjoyed our conversation the oth...,3
4,'You're fired That's another silly misconcepti...,4


In [18]:
# Remove rows, where the label is present only ones (can't be split)
data = data.groupby('label').filter(lambda x : len(x) > 1)
#data = data.groupby('Product').filter(lambda x : len(x) > 1)

In [19]:
data.head()

,posts,label
0,' enfp and intj moments sportscenter not top ...,0
1,'I'm finding the lack of me in these posts ver...,1
2,'Good one Of course to which I say I know th...,2
3,'Dear INTP I enjoyed our conversation the oth...,3
4,'You're fired That's another silly misconcepti...,4


In [20]:
# shaffle datasets
data = data.sample(frac=1).reset_index(drop=True)
data

,posts,label
0,'Well I would recommend talkingi than practice...,5
1,'Im a little bored and wanted to start a rando...,3
2,'Ya girl if she is a prima donna like you've d...,1
3,'How the passion is dying because of my financ...,3
4,'Women crushes Kristi Yamaguchi Rachel Weisz P...,6
...,...,...
8670,' your favorite place in the whole town is th...,6
8671,'Hellish most certainly in fact i would most l...,0
8672,'estj Yes I like travelling I've been to Ameri...,8
8673,'Wracking my brain and I don't think this is a...,5


In [21]:
data=data.dropna()                                                           # Drop NaN valuues, if any
data=data.reset_index(drop=True)                                             # Reset index after dropping the columns/rows with NaN values
data = shuffle(data)                                                         # Shuffle the dataset
print('Available labels: ',data.label.unique())                              # Print all the unique labels in the dataset
data['posts']=data['posts'].map(preprocess_sentence)                           # Clean the text column using preprocess_sentence function defined above

Available labels:  [ 1  9  8 11 12  2  6  7  3  0  4  5 10 14 13 15]


In [22]:

print('File has {} rows and {} columns'.format(data.shape[0],data.shape[1]))
data.head()

File has 8675 rows and 2 columns


,posts,label
7768,see thread entp forum reason first listen half...,1
8128,martian andy weir good istp got makeup today t...,9
7426,actually girls happy dating year started going...,8
4474,even religion evil perpetuates narrow minded s...,11
7195,isfp looking advice estjs estj friend performa...,8


In [23]:
num_classes=len(data.label.unique())
num_classes

16

In [30]:
from transformers import BertForSequenceClassification
from transformers import TFBertForSequenceClassification

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:

sent= 'how to train the model, lets look at how a trained model calculates its prediction.'
tokens=bert_tokenizer.tokenize(sent)
print(tokens)

['how', 'to', 'train', 'the', 'model', ',', 'lets', 'look', 'at', 'how', 'a', 'trained', 'model', 'calculate', '##s', 'its', 'prediction', '.']


In [32]:
tokenized_sequence= bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =30,pad_to_max_length = True,
return_attention_mask = True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\cacki\anaconda3\envs\py36TorchTf\lib\site-packages\transformers\tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [33]:

tokenized_sequence

{'input_ids': [101, 2129, 2000, 3345, 1996, 2944, 1010, 11082, 2298, 2012, 2129, 1037, 4738, 2944, 18422, 2015, 2049, 17547, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [34]:
bert_tokenizer.decode(tokenized_sequence['input_ids'])

'[CLS] how to train the model, lets look at how a trained model calculates its prediction. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [37]:

sentences=data['posts']
labels=data['label']
len(sentences),len(labels)

(8675, 8675)

In [38]:
input_ids=[] # 입력데디터 리스트 선언
attention_masks=[] # 앞뒤 정보를 비교하여 마스크 예측하기 위한 리스트 선언

for sent in sentences: # 문장하나씩 가져와서 인풋, 마스스크를 추가
    # 문장을 숫자로 바꾼다. 짧은 문장은 패딩처리하고, 토큰화 한다.
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

In [39]:
input_ids

array([[  101,  2156, 11689, ..., 22540,  2003,   102],
       [  101, 20795,  5557, ...,  4699,  4994,   102],
       [  101,  2941,  3057, ...,  9297,  2126,   102],
       ...,
       [  101, 10166,  4333, ...,  2147,  3849,   102],
       [  101,  5702, 13869, ..., 28653,  3431,   102],
       [  101,  2442,  6449, ...,  4312,  2036,   102]])

In [40]:
attention_masks

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [41]:
labels

array([1, 9, 8, ..., 6, 2, 3], dtype=int64)

In [42]:
len(input_ids),len(attention_masks),len(labels)

(8675, 8675, 8675)

In [43]:
print('Preparing the pickle file.....')

pickle_inp_path='./data_pc/bert_inp.pkl'
pickle_mask_path='./data_pc/bert_mask.pkl'
pickle_label_path='./data_pc/bert_label.pkl'

pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))



Preparing the pickle file.....


In [44]:
print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Loading the saved pickle files..
Input shape (8675, 64) Attention mask shape (8675, 64) Input label shape (8675,)


In [45]:
input_ids # 입력문장

array([[  101,  2156, 11689, ..., 22540,  2003,   102],
       [  101, 20795,  5557, ...,  4699,  4994,   102],
       [  101,  2941,  3057, ...,  9297,  2126,   102],
       ...,
       [  101, 10166,  4333, ...,  2147,  3849,   102],
       [  101,  5702, 13869, ..., 28653,  3431,   102],
       [  101,  2442,  6449, ...,  4312,  2036,   102]])

In [46]:
attention_masks # 마스크처리된 데이터


array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [47]:
labels # 문장당 레이블(5 가지 척도)

array([1, 9, 8, ..., 6, 2, 3], dtype=int64)

In [48]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

Train inp shape (6940, 64) Val input shape (1735, 64)
Train label shape (6940,) Val label shape (1735,)
Train attention mask shape (6940, 64) Val attention mask shape (1735, 64)


In [49]:
# 예측에 사용하는 데이터

# input_ids : val_inp    --> 입력문장
# val_mask --> 마스크처리된 단어

In [50]:
val_inp

array([[  101, 26478,  8609, ...,  4465, 11895,   102],
       [  101,  2817, 23347, ...,  2360,  2111,   102],
       [  101,  3071,  3531, ...,  2094,  2190,   102],
       ...,
       [  101, 14145,  2080, ...,  2111,  4276,   102],
       [  101,  5262,  5852, ...,  2627,  2086,   102],
       [  101,  4169,  2088, ...,  3501,  3595,   102]])

In [51]:
val_inp.shape

(1735, 64)

In [52]:
val_label

array([7, 3, 0, ..., 1, 9, 2], dtype=int64)

In [53]:
val_label.shape

(1735,)

In [54]:
val_mask

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [55]:
val_mask.shape

(1735, 64)

In [56]:
log_dir='tensorboard_data/tb_bert'
model_save_path='./models/bert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

print('\nBert Model',bert_model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  12304     
Total params: 109,494,544
Trainable params: 109,494,544
Non-trainable params: 0
_________________________________________________________________

Bert Model None


In [ ]:

history=bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=100,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/100
217/217 [==============================] - 47s 218ms/step - loss: 1.6533 - accuracy: 0.5027 - val_loss: 2.0530 - val_accuracy: 0.3873
Epoch 2/100
217/217 [==============================] - 47s 216ms/step - loss: 1.3396 - accuracy: 0.6157 - val_loss: 2.1699 - val_accuracy: 0.3787
Epoch 3/100
217/217 [==============================] - 47s 217ms/step - loss: 1.0268 - accuracy: 0.7122 - val_loss: 2.3469 - val_accuracy: 0.3683
Epoch 4/100
217/217 [==============================] - 48s 219ms/step - loss: 0.7508 - accuracy: 0.7938 - val_loss: 2.6685 - val_accuracy: 0.3487
Epoch 5/100
217/217 [==============================] - 47s 219ms/step - loss: 0.5256 - accuracy: 0.8646 - val_loss: 2.8910 - val_accuracy: 0.3343
Epoch 6/100
141/217 [==================>...........] - ETA: 15s - loss: 0.3870 - accuracy: 0.8980

In [ ]:

model_save_path='./models/bert_model.h5'

trained_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=16)
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

preds = trained_model.predict([val_inp,val_mask],batch_size=32)
print('preds:', preds)
print(type(preds))
preds = preds['logits']
print('preds_:', preds)
# import numpy as np
# pred_labels = np.argmax(preds)
pred_labels = preds.argmax(axis=1)
print('pred_labels', pred_labels)
print('val_label', val_label)

f1 = f1_score(val_label,pred_labels, average='micro')
print('F1 score',f1)
print('Classification Report')


target_names = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP', 'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']
print(classification_report(val_label,pred_labels,target_names=target_names))

print('Training and saving built model.....')

In [ ]:

# sentnece label : lean_left

sent_input = """One Metropolitan Police officer who was beaten and tased during the riot, Michael Fanone, has advocated for the release of his body camera video and repeatedly criticized Republicans who've tried to downplay the violence during the insurrection."""



# 문장을 숫자로 바꾼다. 짧은 문장은 패딩처리하고, 토큰화 한다.
bert_inp=bert_tokenizer.encode_plus(sent_input,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
input_ids=bert_inp['input_ids']
attention_masks= bert_inp['attention_mask']

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)


In [ ]:
bert_inp

In [ ]:
input_ids

In [ ]:
attention_masks

In [ ]:
# val_inp : input_ids
# val_mask : attention_masks

preds = trained_model.predict([input_ids,attention_masks],batch_size=32)
preds = preds['logits'] # 시그모이드함수 적용하여 데이터의 결과를 두 그룹으로 분류해본다. 0 or 1
print("prieds'logit':", preds)
pred_labels = preds.argmax(axis=1)
print('pred_labels', pred_labels)

In [ ]:
type(pred_labels)

In [ ]:
plt.hist(pred_labels)

plt.title("MBTI AI")


plt.show()

In [ ]:
result = np.bincount(pred_labels)
result = np.argmax(result)
result

In [ ]:
{'INFJ': 0,
 'ENTP': 1,
 'INTP': 2,
 'INTJ': 3,
 'ENTJ': 4,
 'ENFJ': 5,
 'INFP': 6,
 'ENFP': 7,
 'ISFP': 8,
 'ISTP': 9,
 'ISFJ': 10,
 'ISTJ': 11,
 'ESTP': 12,
 'ESFP': 13,
 'ESTJ': 14,
 'ESFJ': 15}

In [ ]:
def get_mbti(result):
    if result == 0:
        print('INFJ')
        result = 'INFJ'
    elif result == 1:
        print('ENTP')
        result = 'ENTP'
    elif result == 2:
        print('INTP')
        result = 'INTP'
    elif result == 3:
        print('INTJ')
        result = 'INTJ'
    elif result == 4:
        print('ENTJ')
        result = 'ENTJ'
    elif result == 5:
        print('ENFJ')
        result = 'ENFJ'
    elif result == 6:
        print('INFP')
        result = 'INFP'
    elif result == 7:
        print('ENFP')
        result = 'ENFP'
    elif result == 8:
        print('ISFP')
        result = 'ISFP'
    elif result == 9:
        print('ISTP')
        result = 'ISTP'
    elif result == 10:
        print('ISFJ')
        result = 'ISFJ'
    elif result == 11:
        print('ISTJ')
        result = 'ISTJ'
    elif result == 12:
        print('ESTP')
        result = 'ESTP'
    elif result == 13:
        print('ESFP')
        result = 'ESFP'
    elif result == 14:
        print('ESTJ')
        result = 'ESTJ'
    else:# reulst == 15:
        print('ESFJ')
        result = 'ESFJ'

    return result

In [ ]:
Final_Result = get_mbti(result)